In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tflearn
from sklearn.model_selection import train_test_split

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
df=pd.read_json('reddit_jokes.json')

In [3]:
df.head()

,body,id,score,title
0,"Now I have to say ""Leroy can you please paint ...",5tz52q,1,I hate how you cant even say black paint anymore
1,Pizza doesn't scream when you put it in the ov...,5tz4dd,0,What's the difference between a Jew in Nazi Ge...
2,...and being there really helped me learn abou...,5tz319,0,I recently went to America....
3,A Sunday school teacher is concerned that his ...,5tz2wj,1,"Brian raises his hand and says, “He’s in Heaven.”"
4,He got caught trying to sell the two books to ...,5tz1pc,0,You hear about the University book store worke...


In [4]:
text=df['body'].values

In [5]:
import re
def normalize_text(text):

    text=text.lower()
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+)|(pic\.twitter\.com/[^\s]+))','', text)
    text = re.sub('@[^\s]+','', text)
    text = re.sub('#([^\s]+)', '', text)
    text = re.sub('[:;>?<=*+()&,\-#!$%\{˜|\}\[^_\\@\]1234567890’‘]',' ', text)
    text = re.sub('[\d]','', text)
    text = text.replace(".", '')
    text = text.replace("'", '')
    text = text.replace("`", '')
    text = text.replace("'s", '')
    text = text.replace("/", ' ')
    text = text.replace("\"", ' ')
    text = text.replace("\\", '')
    re.sub(' +', ' ', text)
    text=text.replace('\n', ' ').replace('\r', ' ').replace('\t', ' ')
    #normalize some utf8 encoding
    text = text.replace("\x9d",'').replace("\x8c",'')
    text = text.replace("\xa0",'')
    text = text.replace("\x9d\x92", '').replace("\x9a\xaa\xf0\x9f\x94\xb5", '').replace("\xf0\x9f\x91\x8d\x87\xba\xf0\x9f\x87\xb8", '').replace("\x9f",'').replace("\x91\x8d",'')
    text = text.replace("\xf0\x9f\x87\xba\xf0\x9f\x87\xb8",'').replace("\xf0",'').replace('\xf0x9f','').replace("\x9f\x91\x8d",'').replace("\x87\xba\x87\xb8",'')
    text = text.replace("\xe2\x80\x94",'').replace("\x9d\xa4",'').replace("\x96\x91",'').replace("\xe1\x91\xac\xc9\x8c\xce\x90\xc8\xbb\xef\xbb\x89\xd4\xbc\xef\xbb\x89\xc5\xa0\xc5\xa0\xc2\xb8",'')
    text = text.replace("\xe2\x80\x99s", "").replace("\xe2\x80\x98", '').replace("\xe2\x80\x99", '').replace("\xe2\x80\x9c", "").replace("\xe2\x80\x9d", "")
    text = text.replace("\xe2\x82\xac", "").replace("\xc2\xa3", "").replace("\xc2\xa0", "").replace("\xc2\xab", "").replace("\xf0\x9f\x94\xb4", "").replace("\xf0\x9f\x87\xba\xf0\x9f\x87\xb8\xf0\x9f", "")
    text =  re.sub(r"\b[a-z]\b", "", text)
    text=re.sub( '\s+', ' ', text).strip()
    
    text=re.sub(r'\.+', ".", text)
    text=re.sub(r'\.\.+', ' ', text).replace('.', '')
    # Replace multiple dots with space
    text = re.sub('\.\.+', ' ', text) 
    # Remove single dots
    text = re.sub('\.', '', text)
    text = re.sub(r'\.{2,}', ' ', text)
    text = re.sub(r'\.{1}', '', text)
    
    return text

In [7]:
normed_text=[]
for sentence in df['body']:
    normed_text.append(normalize_text(sentence))

In [8]:
max_sent_length = 50
#make vocab processor
vocab_processor = tflearn.data_utils.VocabularyProcessor(max_sent_length)
# our sentences represented as indices instead of words
data = list(vocab_processor.fit_transform(normed_text))

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.


In [9]:
vocab_size = len(vocab_processor.vocabulary_)
#convert numpy lists to python lists
data = [i.tolist() for i in data]
##get rid of trailing 0s
for lst in data:
    try:
        ##pop off last index if it is equal to 0
        while lst[-1] == 0:
            lst.pop()
    except:
        pass
#filter the empty lists
data = filter(None, data)
#convert data to numpy list after converting filter to python list
data = np.array(list(data))

In [10]:
window=2
pivot_words = []
##target words i.e. our y's or outputs
target_words = []
# data shape = [num sentences, sentence_length]
for d in range(data.shape[0]):
    pivot_idx = data[d][window:-window]

    for i in range(len(pivot_idx)):
        #get the current pivot word
        pivot = pivot_idx[i]

        #targets array
        targets = np.array([])

        neg_target = data[d][i : window+i]
        pos_target = data[d][i + window +1: i + window + window +1]
        targets = np.append(targets, [neg_target, pos_target]).flatten().tolist()

        for c in range(window*2):
            pivot_words.append(pivot)
            target_words.append(targets[c])

In [11]:
def word_to_vec():
    embed_size=128
    num_samples=64
    learning_rate = 0.001
    x=tf.placeholder(shape=[None,], dtype=tf.int32, name="x_center_id")
    y=tf.placeholder(shape=[None,], dtype=tf.int32, name="y_target_id")
    Embedding = tf.Variable(tf.random_uniform([vocab_size,embed_size],-1.0,1.0), name="word_embeddings")
    nce_weights= tf.Variable(tf.truncated_normal([vocab_size, embed_size],stddev=tf.sqrt(1/embed_size)), name="nce_weights")
    nce_baises = tf.Variable(tf.zeros([vocab_size]), name="nce_bias")
    pivot = tf.nn.embedding_lookup(Embedding, x, name="embedding_lookup")
    train_labels= tf.reshape(y, [tf.shape(y)[0], 1])
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights, biases=nce_baises, labels=train_labels, inputs=pivot, num_sampled=num_samples, num_classes=vocab_size, num_true=1))
    optimizer = tf.contrib.layers.optimize_loss(loss,tf.train.get_global_step(),learning_rate,"Adam",clip_gradients=5.0,name="Optimizer")

    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    return optimizer, loss, x, y, sess

In [12]:
optimizer, loss, x, y, sess = word_to_vec()

X_train, X_test, y_train, y_test = train_test_split(pivot_words, target_words)

batch_size = 32
num_epochs = 5

#Num batches in training set
num_batches = len(X_train) // batch_size

#create saver to save our weights
saver = tf.train.Saver()

for e in range(num_epochs):
    for i in range(num_batches):
        if i != range(num_batches-1):
            x_batch = X_train[i*batch_size:i * batch_size + batch_size]
            y_batch = y_train[i*batch_size:i * batch_size + batch_size]
        else:
            x_batch = X_train[i*batch_size:]
            y_batch = y_train[i*batch_size:]

        _, l = sess.run([optimizer, loss], feed_dict = {x: x_batch, y: y_batch})

        if i>0 and i %100 == 0:
            print("STEP", i, "of", num_batches, "LOSS:", l)
    save_path = saver.save(sess, "./word2vec_model.ckpt")

STEP 100 of 237859 LOSS: 266.82706
STEP 200 of 237859 LOSS: 319.50702
STEP 300 of 237859 LOSS: 306.35516
STEP 400 of 237859 LOSS: 329.1489
STEP 500 of 237859 LOSS: 291.06677
STEP 600 of 237859 LOSS: 225.70999
STEP 700 of 237859 LOSS: 238.36084
STEP 800 of 237859 LOSS: 232.93266
STEP 900 of 237859 LOSS: 237.52977
STEP 1000 of 237859 LOSS: 234.98282
STEP 1100 of 237859 LOSS: 289.67047
STEP 1200 of 237859 LOSS: 226.59872
STEP 1300 of 237859 LOSS: 227.94543
STEP 1400 of 237859 LOSS: 227.51407
STEP 1500 of 237859 LOSS: 248.52731
STEP 1600 of 237859 LOSS: 198.86066
STEP 1700 of 237859 LOSS: 207.38565
STEP 1800 of 237859 LOSS: 194.64322
STEP 1900 of 237859 LOSS: 205.9089
STEP 2000 of 237859 LOSS: 199.56845
STEP 2100 of 237859 LOSS: 182.57047
STEP 2200 of 237859 LOSS: 171.89236
STEP 2300 of 237859 LOSS: 156.3905
STEP 2400 of 237859 LOSS: 189.57378
STEP 2500 of 237859 LOSS: 158.80524
STEP 2600 of 237859 LOSS: 152.67465
STEP 2700 of 237859 LOSS: 151.87714
STEP 2800 of 237859 LOSS: 163.84314
STEP

STEP 22800 of 237859 LOSS: 40.075245
STEP 22900 of 237859 LOSS: 34.77539
STEP 23000 of 237859 LOSS: 27.884123
STEP 23100 of 237859 LOSS: 17.454952
STEP 23200 of 237859 LOSS: 29.362856
STEP 23300 of 237859 LOSS: 44.803795
STEP 23400 of 237859 LOSS: 9.733752
STEP 23500 of 237859 LOSS: 38.45448
STEP 23600 of 237859 LOSS: 28.104227
STEP 23700 of 237859 LOSS: 26.94838
STEP 23800 of 237859 LOSS: 21.31338
STEP 23900 of 237859 LOSS: 26.210938
STEP 24000 of 237859 LOSS: 17.437284
STEP 24100 of 237859 LOSS: 36.240574
STEP 24200 of 237859 LOSS: 26.413656
STEP 24300 of 237859 LOSS: 18.10022
STEP 24400 of 237859 LOSS: 26.563194
STEP 24500 of 237859 LOSS: 21.894041
STEP 24600 of 237859 LOSS: 8.674709
STEP 24700 of 237859 LOSS: 18.455818
STEP 24800 of 237859 LOSS: 36.277405
STEP 24900 of 237859 LOSS: 30.114204
STEP 25000 of 237859 LOSS: 24.525135
STEP 25100 of 237859 LOSS: 23.61957
STEP 25200 of 237859 LOSS: 19.490595
STEP 25300 of 237859 LOSS: 34.403687
STEP 25400 of 237859 LOSS: 20.544834
STEP 2550

STEP 45200 of 237859 LOSS: 6.328998
STEP 45300 of 237859 LOSS: 14.049558
STEP 45400 of 237859 LOSS: 8.131068
STEP 45500 of 237859 LOSS: 12.916044
STEP 45600 of 237859 LOSS: 10.201239
STEP 45700 of 237859 LOSS: 19.094028
STEP 45800 of 237859 LOSS: 10.534662
STEP 45900 of 237859 LOSS: 7.0801635
STEP 46000 of 237859 LOSS: 8.7791605
STEP 46100 of 237859 LOSS: 12.145319
STEP 46200 of 237859 LOSS: 19.686707
STEP 46300 of 237859 LOSS: 5.8130836
STEP 46400 of 237859 LOSS: 10.713291
STEP 46500 of 237859 LOSS: 18.538212
STEP 46600 of 237859 LOSS: 9.202403
STEP 46700 of 237859 LOSS: 14.320734
STEP 46800 of 237859 LOSS: 9.684617
STEP 46900 of 237859 LOSS: 15.417639
STEP 47000 of 237859 LOSS: 5.9926124
STEP 47100 of 237859 LOSS: 21.151455
STEP 47200 of 237859 LOSS: 5.41206
STEP 47300 of 237859 LOSS: 13.184311
STEP 47400 of 237859 LOSS: 8.946079
STEP 47500 of 237859 LOSS: 15.963872
STEP 47600 of 237859 LOSS: 8.481514
STEP 47700 of 237859 LOSS: 12.8539095
STEP 47800 of 237859 LOSS: 10.984953
STEP 479

STEP 67600 of 237859 LOSS: 12.250902
STEP 67700 of 237859 LOSS: 6.624948
STEP 67800 of 237859 LOSS: 10.603014
STEP 67900 of 237859 LOSS: 7.634921
STEP 68000 of 237859 LOSS: 23.47526
STEP 68100 of 237859 LOSS: 4.048181
STEP 68200 of 237859 LOSS: 8.731743
STEP 68300 of 237859 LOSS: 5.2625036
STEP 68400 of 237859 LOSS: 7.5406055
STEP 68500 of 237859 LOSS: 12.982437
STEP 68600 of 237859 LOSS: 13.163601
STEP 68700 of 237859 LOSS: 3.9313183
STEP 68800 of 237859 LOSS: 11.662472
STEP 68900 of 237859 LOSS: 10.884954
STEP 69000 of 237859 LOSS: 14.814308
STEP 69100 of 237859 LOSS: 6.8969946
STEP 69200 of 237859 LOSS: 18.351187
STEP 69300 of 237859 LOSS: 10.136721
STEP 69400 of 237859 LOSS: 6.0792894
STEP 69500 of 237859 LOSS: 18.797161
STEP 69600 of 237859 LOSS: 8.628956
STEP 69700 of 237859 LOSS: 6.4680643
STEP 69800 of 237859 LOSS: 22.404127
STEP 69900 of 237859 LOSS: 9.078835
STEP 70000 of 237859 LOSS: 9.995186
STEP 70100 of 237859 LOSS: 7.1165094
STEP 70200 of 237859 LOSS: 5.609754
STEP 70300

KeyboardInterrupt: 